In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv(r'E:\Udemy\Machine Learning A-Z\Part 8 - Deep Learning\Section 39 - Artificial Neural Networks (ANN)\Artificial_Neural_Networks\Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [3]:
# Coverting Text into numerical values

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
X_LE = LabelEncoder()
X[:,2] = X_LE.fit_transform(X[:, 2])
ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[: , 1:]
X.shape

(10000, 11)

In [4]:
# Splitting dataset into Training and Testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [6]:
# Implementation of ANN

import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising ANN
classifier = Sequential()

# Adding Input and 1st hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding 2nd hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fittting data
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)



Using TensorFlow backend.
C:\Users\Ram Mandal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ram Mandal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ram Mandal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ram Mandal\anaconda3\lib\site-packages\tensorflow\python

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
8000/8000 [==============================] - 4s 560us/step - loss: 0.4856 - accuracy: 0.7956
Epoch 2/10
8000/8000 [==============================] - 3s 401us/step - loss: 0.4284 - accuracy: 0.7960
Epoch 3/10
8000/8000 [==============================] - 4s 499us/step - loss: 0.4229 - accuracy: 0.7975
Epoch 4/10
8000/8000 [==============================] - 3s 359us/step - loss: 0.4187 - accuracy: 0.8196
Epoch 5/10
8000/8000 [==============================] - 4s 466us/step - loss: 0.4161 - accuracy: 0.8273
Epoch 6/10
8000/8000 [==============================] - 4s 449us/step - loss: 0.4143 - accuracy: 0.8305
Epoch 7/10
8000/8000 [==============================] - 4s 484us/step - loss: 0.4126 - accuracy: 0.8328
Epoch 8/10
8000/8000 [==============================] - 3s 402us/step - loss: 0.4116 - accuracy: 0.8334
Epoch 9/10
8000/8000 [==============================] - 3s 420us/step - l

In [7]:
# Prediction

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [8]:
# Confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1566,   29],
       [ 298,  107]], dtype=int64)

In [9]:
(1533+182)/2000

0.8575

In [10]:
# As Training set contains Accuracy of 85.09% and the Test set contains Accuracy of 85.75%
# Hence Validated

In [11]:
new_pred = classifier.predict(sc_X.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
(new_pred > 0.5)

array([[False]])

In [12]:
# Evaluating ANN by K-Fold Cross Validation

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
accuracies


array([0.83125001, 0.83375001, 0.83499998, 0.82875001, 0.85124999,
       0.83125001, 0.82999998, 0.82125002, 0.815     , 0.84125   ])

In [13]:
# Calc. MEAN and VARIANCE

mean = accuracies.mean()
variance = accuracies.std()
print(f'MEAN = {mean} and VARIENCE = {variance}%')

MEAN = 0.8318750023841858 and VARIENCE = 0.009424863325599643%


In [ ]:
# Tuning ANN

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameter = {'optimizer' : ['adam', 'rmsprop'],
            'batch_size' : [10, 25, 32],
            'epochs' : [20, 30, 40]}
grid_search = GridSearchCV(estimator = classifier,
                          param_grid = parameter,
                          scoring = 'accuracy')
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/20
6400/6400 [==============================] - 3s 432us/step - loss: 0.5107 - accuracy: 0.7970
Epoch 2/20
6400/6400 [==============================] - 3s 427us/step - loss: 0.4296 - accuracy: 0.7980
Epoch 3/20
6400/6400 [==============================] - 2s 330us/step - loss: 0.4256 - accuracy: 0.7980
Epoch 4/20
6400/6400 [==============================] - 2s 326us/step - loss: 0.4218 - accuracy: 0.7980
Epoch 5/20
6400/6400 [==============================] - 3s 427us/step - loss: 0.4189 - accuracy: 0.8050
Epoch 6/20
6400/6400 [==============================] - 2s 381us/step - loss: 0.4165 - accuracy: 0.8228
Epoch 7/20
6400/6400 [==============================] - 3s 458us/step - loss: 0.4148 - accuracy: 0.8277
Epoch 8/20
6400/6400 [==============================] - 2s 374us/step - loss: 0.4136 - accuracy: 0.8313
Epoch 9/20
6400/6400 [==============================] - 2s 330us/step - loss: 0.4124 - accuracy: 0.8294
Epoch 10/20
6400/6400 [==============================] - 3s 418u

6400/6400 [==============================] - 2s 378us/step - loss: 0.4045 - accuracy: 0.8366
Epoch 20/20
6400/6400 [==============================] - 2s 342us/step - loss: 0.4038 - accuracy: 0.8356
Epoch 1/20
6400/6400 [==============================] - 3s 423us/step - loss: 0.4952 - accuracy: 0.7953
Epoch 2/20
6400/6400 [==============================] - 3s 498us/step - loss: 0.4304 - accuracy: 0.7958
Epoch 3/20
6400/6400 [==============================] - 3s 395us/step - loss: 0.4270 - accuracy: 0.7958
Epoch 4/20
6400/6400 [==============================] - 3s 406us/step - loss: 0.4238 - accuracy: 0.7958
Epoch 5/20
6400/6400 [==============================] - 2s 381us/step - loss: 0.4204 - accuracy: 0.8048
Epoch 6/20
6400/6400 [==============================] - 2s 350us/step - loss: 0.4184 - accuracy: 0.8234
Epoch 7/20
6400/6400 [==============================] - 3s 487us/step - loss: 0.4161 - accuracy: 0.8275
Epoch 8/20
6400/6400 [==============================] - 3s 430us/step - lo

6400/6400 [==============================] - 3s 413us/step - loss: 0.4102 - accuracy: 0.8325
Epoch 18/20
6400/6400 [==============================] - 3s 474us/step - loss: 0.4105 - accuracy: 0.8338
Epoch 19/20
6400/6400 [==============================] - 3s 463us/step - loss: 0.4101 - accuracy: 0.8325
Epoch 20/20
6400/6400 [==============================] - 4s 552us/step - loss: 0.4094 - accuracy: 0.8325
Epoch 1/20
6400/6400 [==============================] - 4s 571us/step - loss: 0.5128 - accuracy: 0.7973
Epoch 2/20
6400/6400 [==============================] - 3s 404us/step - loss: 0.4322 - accuracy: 0.7973
Epoch 3/20
6400/6400 [==============================] - 4s 598us/step - loss: 0.4274 - accuracy: 0.7973
Epoch 4/20
6400/6400 [==============================] - 3s 529us/step - loss: 0.4238 - accuracy: 0.7973
Epoch 5/20
6400/6400 [==============================] - 4s 584us/step - loss: 0.4203 - accuracy: 0.8142
Epoch 6/20
6400/6400 [==============================] - 3s 470us/step - 

6400/6400 [==============================] - 2s 369us/step - loss: 0.4167 - accuracy: 0.8177
Epoch 6/30
6400/6400 [==============================] - 2s 377us/step - loss: 0.4120 - accuracy: 0.8322
Epoch 7/30
6400/6400 [==============================] - 2s 356us/step - loss: 0.4092 - accuracy: 0.8319
Epoch 8/30
6400/6400 [==============================] - 3s 406us/step - loss: 0.4060 - accuracy: 0.8358
Epoch 9/30
6400/6400 [==============================] - 3s 433us/step - loss: 0.4042 - accuracy: 0.8341
Epoch 10/30
6400/6400 [==============================] - 3s 392us/step - loss: 0.4022 - accuracy: 0.8350
Epoch 11/30
6400/6400 [==============================] - 3s 397us/step - loss: 0.4009 - accuracy: 0.8370
Epoch 12/30
6400/6400 [==============================] - 2s 368us/step - loss: 0.3997 - accuracy: 0.8359
Epoch 13/30
6400/6400 [==============================] - 2s 365us/step - loss: 0.3984 - accuracy: 0.8358
Epoch 14/30
6400/6400 [==============================] - 3s 404us/step 